# Make HSC export file

The gen 3 Butler does not have a mapper which merely references files by name. files need to be ingested in to the butler. We therefore need to make an export.yaml file containing the appropriate metadata for each HSC file. 

In [1]:
# What version of the Stack are we using?
! eups list -s | grep lsst_distrib
! eups list -s | grep obs_vista

lsst_distrib          g2d4714e03a+6e1aa0b536 	current w_2022_07 w_latest setup
obs_vista             23.0.0-1   	current setup


In [2]:
import glob
import yaml
from astropy.table import Table
import os

In [3]:
calexp = glob.glob('./data/hsc-release.mtk.nao.ac.jp/archive/filetree/pdr3_dud/deepCoadd-results/HSC*/*/*/calexp*.fits')
det = glob.glob('./data/hsc-release.mtk.nao.ac.jp/archive/filetree/pdr3_dud/deepCoadd-results/HSC*/*/*/det*.fits')

In [4]:
os.getcwd()

'/Users/raphaelshirley/Documents/github/lsst-ir-fusion/dmu0/dmu0_HSC'

In [5]:
calexp

['./data/hsc-release.mtk.nao.ac.jp/archive/filetree/pdr3_dud/deepCoadd-results/HSC-R/8524/3,4/calexp-HSC-R-8524-3,4.fits',
 './data/hsc-release.mtk.nao.ac.jp/archive/filetree/pdr3_dud/deepCoadd-results/HSC-R/8524/3,5/calexp-HSC-R-8524-3,5.fits']

In [6]:
det

['./data/hsc-release.mtk.nao.ac.jp/archive/filetree/pdr3_dud/deepCoadd-results/HSC-R/8524/3,4/det-HSC-R-8524-3,4.fits',
 './data/hsc-release.mtk.nao.ac.jp/archive/filetree/pdr3_dud/deepCoadd-results/HSC-R/8524/3,5/det-HSC-R-8524-3,5.fits']

In [7]:
hsc_dict =yaml.safe_load("""
description: Butler Data Repository Export
version: 1.0.1
data:
- type: collection
  collection_type: RUN
  name: videoTest
  host: null
  timespan_begin: null
  timespan_end: null
- type: dataset_type
  name: deepCoadd
  dimensions:
  - band
  - skymap
  - tract
  - patch
  storage_class: ExposureF
  is_calibration: false
- type: dataset
  dataset_type: deepCoadd
  run: videoTest
  records: []
""")



In [8]:
calexp[0]

'./data/hsc-release.mtk.nao.ac.jp/archive/filetree/pdr3_dud/deepCoadd-results/HSC-R/8524/3,4/calexp-HSC-R-8524-3,4.fits'

In [9]:
ims = Table()
ims['filename']=[os.getcwd()+c[1:] for c in calexp]
ims['band']=[c.split('/')[7][-1].lower() for c in calexp]
ims['patch']=[int(c.split('/')[9][2])*9+int(c.split('/')[9][0]) for c in calexp]
ims['physical_filter']=[c.split('/')[7] for c in calexp]
ims['skymap']='hscPdr2'
ims['tract']=[int(c.split('/')[8]) for c in calexp]

In [10]:
ims[:1]

filename,band,patch,physical_filter,skymap,tract
str184,str1,int64,str5,str7,int64
"/Users/raphaelshirley/Documents/github/lsst-ir-fusion/dmu0/dmu0_HSC/data/hsc-release.mtk.nao.ac.jp/archive/filetree/pdr3_dud/deepCoadd-results/HSC-R/8524/3,4/calexp-HSC-R-8524-3,4.fits",r,39,HSC-R,hscPdr2,8524


In [11]:
ims.write('./data/calexp_{}.ecsv'.format(len(ims)),overwrite=True)

In [12]:
tabs = Table()
tabs['filename']=[os.getcwd()+c[1:] for c in det]
tabs['band']=[c.split('/')[7][-1].lower() for c in det]
tabs['patch']=[int(c.split('/')[9][2])*9+int(c.split('/')[9][0]) for c in det]
tabs['physical_filter']=[c.split('/')[7] for c in det]
tabs['skymap']='hscPdr2'
tabs['tract']=[int(c.split('/')[8]) for c in det]

In [13]:
tabs[:1]

filename,band,patch,physical_filter,skymap,tract
str181,str1,int64,str5,str7,int64
"/Users/raphaelshirley/Documents/github/lsst-ir-fusion/dmu0/dmu0_HSC/data/hsc-release.mtk.nao.ac.jp/archive/filetree/pdr3_dud/deepCoadd-results/HSC-R/8524/3,4/det-HSC-R-8524-3,4.fits",r,39,HSC-R,hscPdr2,8524


In [14]:
tabs.write('./data/det_{}.ecsv'.format(len(ims)),overwrite=True)